In [1]:
import pandas as pd
from matplotlib import pyplot as plt
from tabulate import tabulate
import numpy as np
import re
import locale
locale.setlocale(locale.LC_ALL, '')
import sys
import subprocess
from scipy.optimize import minimize
from matplotlib import rcParams
from scipy.optimize import basinhopping
rcParams['figure.figsize'] = 9, 9

In [2]:
samples = pd.read_excel('tabulated_data4.xlsm', sheet_name='Compositions')
sampleNames = list(samples)[1:]
samples

Name McLeod-Mix1 McLeod-Mix2 McLeod-Mix3 McLeod-Mix22  \
0            Group      McLeod      McLeod      McLeod       McLeod   
1           Sample        Mix1        Mix2        Mix3        Mix22   
2               SG    0.705971    0.652476    0.609878     0.698439   
3    Inert Free SG    0.560051    0.558915    0.560063     0.681036   
4              CO2       0.005       0.003      0.0131        0.018   
..             ...         ...         ...         ...          ...   
111            NaN         NaN         NaN         NaN          NaN   
112            NaN         NaN         NaN         NaN          NaN   
113            NaN         NaN         NaN         NaN          NaN   
114            NaN         NaN         NaN         NaN          NaN   
115            NaN         NaN         NaN         NaN          NaN   

    McLeod-Mix24 McLeod-Mix25 Simon&Briggs-10PC Simon&Briggs-20PC API 37-Mix2  \
0         McLeod       McLeod      Simon&Briggs      Simon&Briggs      API 37   
1          Mix24        Mix25              10PC              20PC        Mix2   
2       0.741612      0.59657          0.616048          0.678315    0.789937   
3       0.736766     0.592865           0.55378           0.55378    0.718099   
4         0.0061        0.004                 0                 0      0.0209   
..           ...          ...               ...               ...         ...   
111          NaN          NaN               NaN               NaN         NaN   
112          NaN          NaN               NaN               NaN         NaN   
113          NaN          NaN               NaN               NaN         NaN   
114          NaN          NaN               NaN               NaN         NaN   
115          NaN          NaN               NaN               NaN         NaN   

     ... Custom-A6ST1_2000 Custom-A6ST1_1500 Custom-A6ST1_1000  \
0    ...            Custom            Custom            Custom   
1    ...        A6ST1_2000        A6ST1_1500        A6ST1_1000   
2    ...          1.129789          1.132206          1.164653   
3    ...          0.851054          0.835685          0.867391   
4    ...             0.415             0.432            0.4552   
..   ...               ...               ...               ...   
111  ...               NaN               NaN               NaN   
112  ...               NaN               NaN               NaN   
113  ...               NaN               NaN               NaN   
114  ...               NaN               NaN               NaN   
115  ...               NaN               NaN               NaN   

    Custom-A6ST1_500 Custom-D1_4088 Custom-D1_4089 Custom-D1_4848  \
0             Custom         Custom         Custom         Custom   
1          A6ST1_500        D1_4088        D1_4089        D1_4848   
2             1.2437       1.463583       0.949258       0.880221   
3            0.98819       1.465396       0.942719       0.872138   
4             0.4813        0.01068        0.01118        0.01157   
..               ...            ...            ...            ...   
111              NaN            NaN            NaN            NaN   
112              NaN            NaN            NaN            NaN   
113              NaN            NaN            NaN            NaN   
114              NaN            NaN            NaN            NaN   
115              NaN            NaN            NaN            NaN   

    Custom-D1_4469 Li_Ja_Wu-Sample1 Li_Ja_Wu-Sample2  
0           Custom         Li_Ja_Wu         Li_Ja_Wu  
1          D1_4469          Sample1          Sample2  
2         1.480842         0.572925         0.663684  
3         1.484551         0.571767         0.634273  
4          0.01457                0          0.02853  
..             ...              ...              ...  
111            NaN              NaN              NaN  
112            NaN              NaN              NaN  
113            NaN              NaN              NaN  
114            N

In [3]:
filename ='HC_BIPS_VSHIFT_v01'
Phz_loc = "C:\\Temp\\PhazeComp\\phazecomp.exe"         # The location of the PhazeComp executable on your system
#Phz_loc = "C:\\Program Files\\PhazeComp\\1.81\\phazecomp.exe"         # The location of the PhazeComp executable on your system

phz_template = filename+'.phz' # The .Phz file to use as the base
R, mwAir, degF2R = 10.731577089016, 28.97, 459.67


def make_valid_filename(input_string):
    valid_filename = re.sub(r'[^\w.-]', '_', input_string)
    valid_filename = valid_filename.strip()
    valid_filename = re.sub(r'[-_.]{2,}', '_', valid_filename)
    valid_filename = valid_filename.replace('@', '_').replace('#', '_')
    return valid_filename

def create_phz(phz_template):
    rms=float("nan")
    try:
        proc = subprocess.run([Phz_loc, phz_template,'NUL',">"], capture_output=True)
    except:
        print("Either phazecomp.exe couldnt be found (is it in "+Phz_loc+" ?), or "+phz_template+" couldn't be found - Exiting..")
        sys.exit()
    
    found_rms = False
    for i in range(len(proc.stderr.splitlines())-1,0,-1):
        line = proc.stderr.splitlines()[i].decode("utf-8")
        if "% RMS average weighted" in line:
            rms = float(line.split()[2])
            found_rms = True
            break
    if not found_rms:
        print("Error: No RMS for ",phz_template)
    return rms

## Export all the experiments

In [4]:
mw_hcs = [16.043, 27.91, 34.51, 41.63]    

sample_dic = {}
experiment_list = []
for c, sample in enumerate(sampleNames):                                                                  
    name = sampleNames[c]                                                                           
    group = samples.iloc[0, c+1]                                                                    
    sample = samples.iloc[1, c+1]                                                                   
    sg = samples.iloc[2, c+1]                                                                       
    co2 = samples.iloc[4, c+1]                                                                      
    h2s = samples.iloc[5, c+1]                                                                      
    n2 = samples.iloc[6, c+1]                                                                       
    gas = 1 - co2 - h2s - n2
    experiments = pd.read_excel('tabulated_data4.xlsm', sheet_name=name)
    degfs = list(set(experiments['degf'].to_list()))
    degfs.sort()
    exps = []
    for degf in degfs:
        fitered_exp = experiments[experiments['degf']==degf]
        degf = int(round(degf,0))
        out_str = 'CCE ID "'+name+'-'+str(degf)+'"\n'
        exp_df = pd.DataFrame()
        temps = fitered_exp['degf'].to_list()+['Wt']
        press = fitered_exp['psia'].to_list()+['']
        zs = fitered_exp['Z'].to_list()+[1]
        gss = [1 for z in zs]
        gss[-1] = 0
            
        exp_df['TEMP (F)']= temps
        exp_df['PRES (psia)']= press
        exp_df['Z']= zs
        exp_df['GS']= gss

        out_str += tabulate(exp_df, headers='keys', showindex='never', numalign='left')
        out_str += '\nEND\n\n'
        filename = make_valid_filename(name+'-'+str(degf)+'.inc')
        with open('.\\experiments\\'+filename, 'w') as file:                                   
            #exps.append('.\\experiments\\'+filename)
            exps.append(filename)
            file.write(out_str) 
    print('INCLUDE ".\\experiments\\'+make_valid_filename(name+'.inc')+'"')   
    sample_dic[name] = [name, group, sample, sg, co2, h2s, n2, 0, gas, exps, degfs]   
 

INCLUDE ".\experiments\McLeod-Mix1.inc"
INCLUDE ".\experiments\McLeod-Mix2.inc"
INCLUDE ".\experiments\McLeod-Mix3.inc"
INCLUDE ".\experiments\McLeod-Mix22.inc"
INCLUDE ".\experiments\McLeod-Mix24.inc"
INCLUDE ".\experiments\McLeod-Mix25.inc"
INCLUDE ".\experiments\Simon_Briggs-10PC.inc"
INCLUDE ".\experiments\Simon_Briggs-20PC.inc"
INCLUDE ".\experiments\API_37-Mix2.inc"
INCLUDE ".\experiments\API_37-Mix17.inc"
INCLUDE ".\experiments\API_37-Mix25.inc"
INCLUDE ".\experiments\API_37-Mix26.inc"
INCLUDE ".\experiments\API_37-Mix402.inc"
INCLUDE ".\experiments\API_37-Mix601.inc"
INCLUDE ".\experiments\API_37-Mix705.inc"
INCLUDE ".\experiments\API_37-Mix707.inc"
INCLUDE ".\experiments\API_37-Mix722.inc"
INCLUDE ".\experiments\API_37-Mix727.inc"
INCLUDE ".\experiments\API_37-Mix729.inc"
INCLUDE ".\experiments\API_37-Mix735.inc"
INCLUDE ".\experiments\Satter_Campbell-MixA.inc"
INCLUDE ".\experiments\Satter_Campbell-MixB.inc"
INCLUDE ".\experiments\Satter_Campbell-MixC.inc"
INCLUDE ".\experime

## Script to create CHR's and link to experiements 

In [11]:
mw_ch4, tc_ch4, pc_ch4, zc_ch4, vc_ch4 = 16.043, 343.008, 667.029, 0.286203243, 1.579420001
vc_on_zc_ch4 = vc_ch4 / zc_ch4
comps = ['CO2', 'H2S', 'N2', 'H2', 'Gas']
base_chr = 'RealGas'

def tc_pc(mw, offset, vl, pl, v1, v2, v3):

    vc_on_zc = v1 * mw**2 + v2*mw + v3
    tc = (offset + vc_on_zc) * vl / (offset + vc_on_zc + pl) 
    pc = tc * R / vc_on_zc

    return tc, pc

def write_includes(x, *args):
    x = denorm_solx(x, bounds)
    
    v1, v2, offset, pl = x

    # Anchor Vc/Zc to Methane. Vc/Zc = A * mw ** 2 + B * mw + C
    v3 = vc_on_zc_ch4 - v1 * mw_ch4 ** 2 - v2 * mw_ch4
      
    # Anchor VL to honor methane Tc. Tc = (Offset + (vc/zc)) * VL / (Offset + (vc/zc) + Pl)
    vl = (tc_ch4 *  (offset + vc_on_zc_ch4 + pl))/(offset + vc_on_zc_ch4) 

    sampleNames, sample_dic = args
    SSHIFT = np.array([-0.133582, -0.165534, -0.188442, -0.379055, -1.06936e-01])
    VSHIF2 = np.array([-0.157094159, -0.095827509, -0.008056865, 0.001064581, -0.0737725])
    tcs = np.array([547.416, 672.120, 227.160, 47.430, 1])

    # Wichert Samples
    for name in sampleNames:
        name, group, sample, sg, y_co2, y_h2s, y_n2, y_h2, y_gas, exps, degfs = sample_dic[name]
        zi = [y_co2, y_h2s, y_n2, y_h2, y_gas]
        mws = np.array([44.01, 34.082, 28.014, 2.016, 0])
        sg_hc = (sg - (y_co2 * mws[0] + y_h2s * mws[1] + y_n2 * mws[2] + y_h2 * mws[3]) / mwAir) / (1 - y_co2 - y_h2s - y_n2 - y_h2)
        mw_gas = sg_hc * mwAir
        mws[-1] = mw_gas
        
        tc, pc = tc_pc(mw_gas, offset, vl, pl, v1, v2, v3)

        tcs[-1] = tc
        
        # Parent characterization with MW calculated and Tc, Pc specified
        out_str = 'DEFINE MW '+str(mw_gas)+'\n'
        out_str += 'CHARACTERIZATION "CHR_'+name+'" BASE "'+base_chr+'"\n'
        header = ['COMPONENT', 'MW', 'TC,R', 'PC,PSIA', 'AF', 'VTRAN', 'A', 'B']
        rows = []
        for c in range(len(comps)):
            rows.append([comps[c], '', '', '', '', '', '', ''])
        rows[-1][1] = round(mw_gas, 4)
        rows[-1][2] = round(tc, 4)
        rows[-1][3] = round(pc, 4)

        out_str += tabulate(rows, header)
        out_str += '\nEND\n\n'

        out_str += 'MIX '+make_valid_filename(name)+'\n'
        rows = []
        for c in range(len(comps)):
            rows.append([comps[c], zi[c]])
        out_str += tabulate(rows)
        out_str += '\nEND\n\n'
        
        for t, temp in enumerate(degfs):
            degR = temp + 459.67
            tr = degR / tcs
            vshift = SSHIFT + VSHIF2 * tr
            out_str += 'DEFINE degR '+str(degR)+'\n\n'
            out_str += 'CHARACTERIZATION "CHR_'+name+'_'+str(temp)+'" BASE "CHR_'+name+'"\n'
            header = ['COMPONENT', 'MW', 'TC,R', 'PC,PSIA', 'AF', 'VTRAN', 'A', 'B']
            rows = []
            for c in range(len(comps)):
                rows.append([comps[c], '', '', '', '', '', '', ''])
            
            #for i, vs in enumerate(vshift):
            #    rows[i][5] = vs
                
            out_str += tabulate(rows, header)
            out_str += '\nEND\n\n'
            
            out_str += 'DEFINE degR_Slope CO2_degR_Slope, DEFINE MW_Slope CO2_MW_Slope, DEFINE Intcpt CO2_Intcpt, DEFINE Inert CO2\n'
            out_str += 'INCLUDE bips_calc.inc\n\n'
            
            out_str += 'DEFINE degR_Slope H2S_degR_Slope, DEFINE MW_Slope H2S_MW_Slope, DEFINE Intcpt H2S_Intcpt, DEFINE Inert H2S\n'
            out_str += 'INCLUDE bips_calc.inc\n\n'
            
            out_str += 'DEFINE degR_Slope N2_degR_Slope, DEFINE MW_Slope N2_MW_Slope, DEFINE Intcpt N2_Intcpt, DEFINE Inert N2\n'
            out_str += 'INCLUDE bips_calc.inc\n\n'
            
            out_str += 'DEFINE degR_Slope H2_degR_Slope, DEFINE MW_Slope H2_MW_Slope, DEFINE Intcpt H2_Intcpt, DEFINE Inert H2\n'
            out_str += 'INCLUDE bips_calc.inc\n\n'
            
            out_str += 'REPLACE BIP of CO2 with H2S by CO2_H2S\n'
            out_str += 'REPLACE BIP of CO2 with N2 by CO2_N2\n'
            out_str += 'REPLACE BIP of CO2 with H2 by CO2_H2\n'
            out_str += 'REPLACE BIP of H2S with N2 by H2S_N2\n'
            out_str += 'REPLACE BIP of H2S with H2 by H2S_H2\n'
            out_str += 'REPLACE BIP of N2 with H2 by N2_H2\n\n'
            
            out_str += 'QIX FEED 1 MOLE '+make_valid_filename(name)+'\n'
            out_str += 'INCLUDE "'+exps[t]+'"\n\n'

        with open('.\\experiments\\'+make_valid_filename(name+'.inc'), 'w') as file:                                 
            file.write(out_str)

    # GERG Synthetic samples
    fileloc = 'HC_BIPS_VSHIFT_v01_GERG'
    mw_hcs = [16.043, 27.91, 34.51, 41.63] # All synthetic mixtures
    names=['lean_gas','leanmid_gas', 'mid_gas', 'rich_gas' ]
    degfs = [60, 90, 120, 150, 180, 220, 250, 280]

    for n, name in enumerate(names):
        tc, pc = tc_pc(mw_hcs[n], offset, vl, pl, v1, v2, v3)
                    
        out_str = 'DEFINE Tc '+str(tc)+'\n'
        out_str += 'DEFINE Pc '+str(pc)+'\n'

        with open('.\\'+fileloc+'\\'+name+'_init.inc', 'w') as file:
            file.write(out_str)
        

    rms = create_phz(phz_template)
    print(x, rms)
    return rms



v1, v2, Offset, Pl  = 8.10544585018431e-05, 0.05675590493647402, 11.188806249822868, 183.5258567664124
x = v1, v2, Offset, Pl
x = norm_x0(x, bounds)
args = sampleNames, sample_dic
write_includes(x, *args)

In [12]:
def norm_x0(x, bounds):
    x0 = []
    for i, bound in enumerate(bounds):
        span = bound[1] - bound[0]
        x0.append((x[i] - bound[0])/span)
    return x0

def denorm_solx(x, bounds):
    x0 = []
    for i, bound in enumerate(bounds):
        span = bound[1] - bound[0]
        x0.append(x[i]*span + bound[0])
    return x0

## **** Next, pin Pc for Methane as well. Then try varied Zc rather than Vc ****

In [14]:
#      v1,          v2,           offset,         pl
x0 = [0.000229975, 0.186415901, -0.032901049, 42.6061669]

bounds = [(0.0001, 0.0004), (0.15, 0.21), (-0.04, -0.02), (30, 50)]
x = norm_x0(x0, bounds)

sol = minimize(write_includes, x, method='Nelder-Mead', args = (sampleNames, sample_dic), bounds=[(0, 1), (0, 1), (0, 1), (0, 1)])

[0.000229975, 0.186415901, -0.032901049, 42.6061669] 0.9034
[0.00023647375, 0.186415901, -0.032901049, 42.6061669] 0.9034
[0.000229975, 0.18823669604999999, -0.032901049, 42.6061669] 0.9055
[0.000229975, 0.186415901, -0.03254610145, 42.6061669] 0.9034
[0.000229975, 0.186415901, -0.032901049, 43.236475245] 0.9035
[0.000233224375, 0.18459510595, -0.032723575225, 42.9213210725] 0.9058
[0.00023078734375000002, 0.187326298525, -0.032856680556250004, 42.684955443125] 0.9039
[0.00023241203124999997, 0.185505503475, -0.03276794366875, 42.842532529375] 0.9041
[0.00023119351562500003, 0.1868710997625, -0.032834496334375005, 42.7243497146875] 0.9035
[0.000232005859375, 0.1859607022375, -0.032790127890625, 42.8031382578125] 0.9036
[0.0002313966015625, 0.18664350038125, -0.032823404223437505, 42.74404685046875] 0.9034
[0.00023393517578125002, 0.186529700690625, -0.032684752836718754, 42.044798530234374] 0.9034
[0.00023294513183593748, 0.18650125076796875, -0.03273882687753907, 42.342717708925775] 0